In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import time
import os
from six.moves import cPickle
from text_utils import TextLoader
from tensorflow.contrib import rnn
from char_rnn_model import Model
import argparse

# Define directories, hyperparameter

In [3]:
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# Data and model checkpoints directories
parser.add_argument('--data_dir', type=str, default='')
parser.add_argument('--save_dir', type=str, default='save')
parser.add_argument('--log_dir', type=str, default='logs')
parser.add_argument('--save_every', type=int, default=1000)
parser.add_argument('--init_from', type=str, default=None)
# Model params
parser.add_argument('--model', type=str, default='lstm')
parser.add_argument('--rnn_size', type=int, default=64)
parser.add_argument('--num_layers', type=int, default=2)
# Optimization
parser.add_argument('--seq_length', type=int, default=50)
parser.add_argument('--batch_size', type=int, default=20)
parser.add_argument('--num_epochs', type=int, default=20)
parser.add_argument('--grad_clip', type=float, default=5.)
parser.add_argument('--learning_rate', type=float, default=0.002)
parser.add_argument('--decay_rate', type=float, default=0.97)
parser.add_argument('--output_keep_prob', type=float, default=1.0)
parser.add_argument('--input_keep_prob', type=float, default=1.0)
args, unknown = parser.parse_known_args()

# Load data using TextLoader object

In [4]:
data_loader = TextLoader(args.data_dir, args.batch_size, args.seq_length)
args.vocab_size = data_loader.vocab_size

loading preprocessed files


# Create your model object

In [5]:
model = Model(args)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


# Training

In [ ]:
with tf.Session() as sess:
    # instrument for tensorboard
    summaries = tf.summary.merge_all()
    writer = tf.summary.FileWriter(
            os.path.join(args.log_dir, time.strftime("%Y-%m-%d-%H-%M-%S")))
    writer.add_graph(sess.graph)

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(tf.global_variables())
    # restore model
    if args.init_from is not None:
        saver.restore(sess, ckpt)
    for e in range(args.num_epochs):
        sess.run(tf.assign(model.lr,
                           args.learning_rate * (args.decay_rate ** e)))
        data_loader.reset_batch_pointer()
        state = sess.run(model.initial_state)
        for b in range(data_loader.num_batches):
            start = time.time()
            x, y = data_loader.next_batch()
            feed = {model.input_data: x, model.targets: y}
            for i, (c, h) in enumerate(model.initial_state):
                feed[c] = state[i].c
                feed[h] = state[i].h

            # instrument for tensorboard
            summ, train_loss, state, _ = sess.run([summaries, model.cost, model.final_state, model.train_op], feed)
            writer.add_summary(summ, e * data_loader.num_batches + b)

            end = time.time()
            print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}"
                  .format(e * data_loader.num_batches + b,
                          args.num_epochs * data_loader.num_batches,
                          e, train_loss, end - start))
            if (e * data_loader.num_batches + b) % args.save_every == 0\
                    or (e == args.num_epochs-1 and
                        b == data_loader.num_batches-1):
                # save for the last result
                checkpoint_path = os.path.join(args.save_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path,
                           global_step=e * data_loader.num_batches + b)
                print("model saved to {}".format(checkpoint_path))

# Generating Text